Adapted tutorial from https://sparkbyexamples.com/spark/spark-read-write-dataframe-parquet-example/

In [5]:
!mkdir /tmp/learn-parquet

mkdir: /tmp/learn-parquet: File exists


In [1]:
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 21:22:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/28 21:22:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# create a sample DF

data = [("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]

rdd = spark.sparkContext.parallelize(data)
df = spark.createDataFrame(rdd, schema=["firstname","middlename","lastname","dob","gender","salary"])
df

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: bigint]

In [6]:
df.write.parquet("/tmp/learn-parquet/people.parquet")

In [7]:
!ls /tmp/learn-parquet/people.parquet

_SUCCESS
part-00000-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
part-00001-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
part-00003-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
part-00005-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
part-00007-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
part-00009-12e20a21-9587-4924-95fc-83094cfa643f-c000.snappy.parquet
